### **RISCO DE FOGO: QUEIMADAS NO CERRADO**
<hr>
<p align="justify">
Este trabalho é referente as tarefas desenvolvidas no Bloco 1. Espera-se desenvolver um projeto capaz de utilizar os comandos aprendidos pela linguagem de programação Python, na disciplina de Aprendizado de Máquina, além de desenvolver um sistema de previsão através de Machine Learning. O grupo desenvolvedor é composto pelas discentes:
<p align="justify">
Isabela Bento Beneti </p>
<p align="justify">
Monyque Karoline de Paula Silva </p>
<p align="justify">
Sofia Baccega C.C. de Oliveira </p>
<p align="justify">
Sophia Figueiredo Michel </p>
<hr>

### **Início:**
<p align="justify">
Em um primento momento, iremos demonstrar o processo de tratamento dos dados obtidos via Database do INPE a respeito dos focos de incêndio ocorridos no Cerrado durante o período Janeiro-Julho(2022). É necessário realizar as padronizações de bibliotecas inicias, como a Pandas, Seaborn e Matplotlib, posteriormente, trabalha-se os dados até a otimização destes para a análise dos atríbutos escolhidos.
</p>

### **Coleta:**

In [161]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, mean_squared_error
from sklearn import preprocessing
from itertools import product
from matplotlib import pyplot as plt
from sklearn import tree

In [135]:
#Dados do Cerrado Brasileiro, armazenados mensalmente via .csv
arquivos = ['janeiro.csv','fev.csv','marco.csv','abril.csv','maio.csv','junho.csv','julho.csv']

In [136]:
def read_csv(list_of_files):
    dfs = []
    for x in list_of_files:
        file = pd.read_csv(x, sep=',')
        dfs.append(file)
    df = pd.concat(dfs).reset_index(drop=True)
        
    return df

In [137]:
meses = read_csv(arquivos)

FileNotFoundError: [Errno 2] No such file or directory: 'janeiro.csv'

In [ ]:
#Dados trabalhados para a eliminação de NaN nas colunas e identificação dos tipos de dados
meses.convert_dtypes()
meses.dropna(how='any', inplace=True)

Eliminado as linhas dos dados não atribuidos(NaN), as desenvolvedoras optaram pelo uso do comando drop, do Pandas, de modo a selecionar as colunas não desejadas do arquvivo .csv, otimizando o processo de análise de dados.

In [ ]:
## Para rodar o código pela primeira vez, descomente essas linhas

meses.drop(columns=['Unnamed: 0','estado', 'satelite', 'municipio', 'pais', 'municipio_id', 'estado_id', 'pais_id', 'bioma'], axis = 1, inplace=True)

In [ ]:
meses_fogo = meses['risco_fogo'] > 0
meses = meses[meses_fogo]

Realizada o processo de seleção de colunas, o próximo passo consiste na identificação dos tipos de dados utilizados em cada coluna e especificamente, em nosso caso, na identificação se dé fato só está sendo utilizado as variáveis numéricas.

In [ ]:
meses.dtypes, meses.shape

In [ ]:
display(meses.describe())

In [ ]:
meses = meses.reset_index()
del meses['index']
display(meses)

Realizada o procedimento de identificação dos tipos de dados existentes no código, inicia-se o processo de normalização de dados. Ela é ocorrente por meio do comando "zscore". Esse processo de normalização é extremamente importante para análises estátisticas, principalmente ao considerar que esse processo é remetente a transformação em escala da distribuição de uma variável para poder fazer comparações entre conjunto de elementos, evitando o efeito de influências.

In [ ]:
#c_maximo = meses['numero_dias_sem_chuva'].max()
#c_minimo = meses['numero_dias_sem_chuva'].min()

#p_maximo = meses['precipitacao'].max()
#p_minimo = meses['precipitacao'].min()

#lat_maximo = meses['lat'].max()
#lat_minimo = meses['lat'].min()

#lon_maximo = meses['lon'].max()
#lon_minimo = meses['lon'].min()

#meses['numero_dias_sem_chuva'] = (meses['numero_dias_sem_chuva'] - c_minimo) / (c_maximo - c_minimo)
#meses['precipitacao'] = (meses['precipitacao'] - p_minimo) / (p_maximo - p_minimo)
#meses['lat'] = (meses['lat'] - lat_minimo) / (lat_maximo - lat_minimo)
#meses['lon'] = (meses['lon'] - lon_minimo) / (lon_maximo - lon_minimo)


#display(meses)

### **Análise de Dados**

De modo a analisar os dados obtidos, identifica-se informações estatísticas sobre cada amostra mensal de dados. Inicialmente, realiza-se o plot da descrição estátisca dos dados, como obter, por exemplo: a média e a moda. Em seguida, realiza-se o plot dos gráficos por meio de uma matriz de covariância e correlação entre os atríbutos para analisar as relações entre os atributos.

In [ ]:
# Matriz de gráficos scatter 
sns.pairplot(meses, height=3);

# mostra o gráfico usando a função show() da matplotlib
plt.show()

In [ ]:
# Matriz de covâriancia
mes_covariancia = meses.cov()

# Matriz de correlação
mes_correlacao = meses.corr()

print('Matriz de covariância: ')
display(mes_covariancia)

print('\n\nMatriz de correlação: ')
display(mes_correlacao)

In [ ]:
labs = ["lat", "lon", "numero_dias_sem_chuva", "precipitacao", "risco_fogo", "data_hora_gmt"]
sns.heatmap(mes_correlacao, annot=True, fmt='g', xticklabels=labs, yticklabels=labs, cmap='seismic', center = 0)
plt.show()

In [ ]:
ax = meses.plot(kind='scatter', x="precipitacao", y="risco_fogo", color='Red', label="Meses")

plt.show()

In [ ]:
meses.plot(kind='hist')

# Bloco 2

In [ ]:
TAMANHO_TESTE = 0.1 # fração de dados utilizada para teste: 10%
SEMENTE_ALEATORIA = 61455 # semente escolhida aleatoriamente
DATASET_NAME = "meses"
FEATURES = ["numero_dias_sem_chuva", "precipitacao", "lat", "lon"]
TARGET = ["risco_fogo"]

indices = meses.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

meses_treino = meses.loc[indices_treino]
meses_teste = meses.loc[indices_teste]

X_treino = meses_treino.reindex(FEATURES, axis=1).values
y_treino = meses_treino.reindex(TARGET, axis=1).values
X_teste = meses_teste.reindex(FEATURES, axis=1).values
y_teste = meses_teste.reindex(TARGET, axis=1).values

In [ ]:
normalizador_x = MinMaxScaler()
normalizador_y = MinMaxScaler()

normalizador_x.fit(X_treino)
normalizador_y.fit(y_treino)

## Baseline

In [ ]:
# cria o modelo
modelo_baseline = DummyRegressor()
# treina o modelo
modelo_baseline.fit(normalizador_x.transform(X_treino),normalizador_y.transform(y_treino))

# realiza uma previsão usando o modelo treinado
previsao = modelo_baseline.predict(normalizador_x.transform(X_treino))
#previsao = normalizador_y.inverse_transform(previsao)
print(previsao)

In [ ]:
y_verdadeiro = y_teste
y_previsao = modelo_baseline.predict(normalizador_x.transform(X_teste))
#y_previsao = normalizador_y.inverse_transform(y_previsao)

RMSE = mean_squared_error(y_verdadeiro, y_previsao, squared=False)

print(f"O RMSE do modelo linear foi de {RMSE}")

# K-vizinhos mais próximos

In [ ]:
# cria o modelo
modelo_knm = KNeighborsRegressor()

# treina o modelo
modelo_knm.fit(
    normalizador_x.transform(X_treino),
    normalizador_y.transform(y_treino),
)

# realiza uma previsão usando o modelo treinado
previsao = modelo_knm.predict(normalizador_x.transform(X_treino))
previsao = normalizador_y.inverse_transform(previsao)
print(previsao)

In [ ]:
NUM_VIZINHOS = [1, 5, 10]

for n in NUM_VIZINHOS:
    modelo_knn = KNeighborsRegressor(n_neighbors=n)

    modelo_knm.fit(normalizador_x.transform(X_treino),normalizador_y.transform(y_treino))
    y_verdadeiro = y_teste
    y_previsao = modelo_knm.predict(normalizador_x.transform(X_teste))
    y_previsao = normalizador_y.inverse_transform(y_previsao)
    RMSE = mean_squared_error(y_verdadeiro, y_previsao, squared=False)
    print(f"O RMSE do modelo linear foi de {RMSE} com {n} vizinhos.")

In [ ]:
# Split into training and test set
neighbors = np.arange(1, 9)
train_accuracy = np.empty(len(neighbors))
test_accuracy = np.empty(len(neighbors))
 
# Loop over K values
for i, k in enumerate(neighbors):
    modelo_knm = KNeighborsRegressor(n_neighbors=k)
    modelo_knm.fit(normalizador_x.transform(X_treino),normalizador_y.transform(y_treino))

    # Compute training and test data accuracy
    train_accuracy[i] = modelo_knm.score(normalizador_x.transform(X_teste), normalizador_y.transform(y_teste))
    test_accuracy[i] = modelo_knm.score(normalizador_x.transform(X_teste), normalizador_y.transform(y_teste))
 
# Plot do Gráfico
plt.plot(neighbors, test_accuracy, label = 'Teste de Precisão do Dataset')
plt.plot(neighbors, train_accuracy, label = 'Treino de Precisão do Dataset')
 
plt.legend()
plt.xlabel('k_vizinhos')
plt.ylabel('Precisão')
plt.show()

# Modelo de Regressão Linear

In [ ]:
# cria o modelo
modelo_linear = LinearRegression()

# treina o modelo
modelo_linear.fit(normalizador_x.transform(X_treino), normalizador_y.transform(y_treino))

# realiza uma previsão usando o modelo treinado
previsao = modelo_linear.predict(normalizador_x.transform(X_treino))
previsao = normalizador_y.inverse_transform(previsao)
print(previsao)

In [ ]:
y_verdadeiro = y_teste
y_previsao = modelo_linear.predict(normalizador_x.transform(X_teste))
y_previsao = normalizador_y.inverse_transform(y_previsao)

RMSE = mean_squared_error(y_verdadeiro, y_previsao, squared=False)

print(f"O RMSE do modelo linear foi de {RMSE}.")

## Árvore de Decisão

In [ ]:
# cria o modelo
modelo_dt = DecisionTreeRegressor(random_state=SEMENTE_ALEATORIA)

# treina o modelo
modelo_dt.fit(normalizador_x.transform(X_treino), normalizador_y.transform(y_treino))

# realiza uma previsão usando o modelo treinado
previsao = modelo_dt.predict(normalizador_x.transform(X_treino))
previsao = normalizador_y.inverse_transform(previsao)
print(previsao)

In [ ]:
y_verdadeiro = y_teste
y_previsao = modelo_dt.predict(normalizador_x.transform(X_teste))
y_previsao = normalizador_y.inverse_transform(y_previsao)

RMSE = mean_squared_error(y_verdadeiro, y_previsao, squared=False)

print(f"O RMSE do modelo linear foi de {RMSE}.")

## Classificação
### com Floresta Aleatória

In [145]:
modelo_rf = RandomForestClassifier(random_seed=SEMENTE_ALEATORIA)

y_treino = y_treino.reshape()
y_teste = y_teste.reshape()

# treina o modelo
modelo_rf.fit(X_treino, y_treino)

# realiza uma previsão usando o modelo treinado
previsao = modelo_rf.predict(X_teste)
print(previsao)

# Criação do Modelo


previsao = modelo_rf.predict(X_teste_norm)

print(previsao)

TypeError: __init__() got an unexpected keyword argument 'random_seed'

#### Métricas:

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score

In [146]:
print(f"A precisão do modelo é de {round(accuracy_score(y_teste,previsao),3)*100} %")

ValueError: continuous is not supported

In [149]:
# sklearn.metrics.precision_score(y_true, y_pred, *, labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')

precision_score(y_true, y_pred, average='macro')
precision_score(y_true, y_pred, average='micro')
precision_score(y_true, y_pred, average='weighted')

SyntaxError: iterable argument unpacking follows keyword argument unpacking (Temp/ipykernel_33004/299440833.py, line 1)

In [148]:
def plot_confusion_matrix(cm, classes, normalize = False,
                          title='Confusion matrix',
                          cmap=plt.cm.Greens): # muda a cor
    plt.figure(figsize = (10, 10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, size = 24),
    plt.colorbar(aspect=4),
    tick_marks = np.arange(len(classes)),
    plt.xticks(tick_marks, classes, rotation=45, size = 14)
    plt.yticks(tick_marks, classes, size = 14),
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.    # Label the plot
    for i, j in itertools.product(range(cm.shape[0]),
        range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), 
        fontsize = 20,
        horizontalalignment="center",
        color="white" if cm[i, j] > thresh else "black"),
    plt.grid(None)
    plt.tight_layout()
    plt.ylabel('True label', size = 18)
    plt.xlabel('Predicted label', size = 18)

In [ ]:
# Plotar como confusion_matrix(y_test, y_pred)
plot_confusion_matrix()

#### HIPERPARÂMETROS

In [ ]:
NUM_FOLHAS = [None, 6, 36]
NUM_PROFUNDIDADE = [None, 3, 10]

for n_folhas, n_profundidade in product(NUM_FOLHAS, NUM_PROFUNDIDADE):
    modelo_dt = DecisionTreeRegressor(
        max_leaf_nodes=n_folhas,
        max_depth=n_profundidade,
        random_state=SEMENTE_ALEATORIA,
    )

    modelo_dt.fit(normalizador_x.transform(X_treino), normalizador_y.transform(y_treino))

    y_verdadeiro = y_teste
    y_previsao = modelo_dt.predict(normalizador_x.transform(X_teste))
    y_previsao = normalizador_y.inverse_transform(y_previsao)
    RMSE = mean_squared_error(y_verdadeiro, y_previsao, squared=False)

    print(
        f"O RMSE do modelo AD usando max_leaf_nodes={n_folhas} "
        f"e max_depth={n_profundidade} foi de {RMSE:.2f}."
    )

#### GRAFO DO MODELO DE ÁRVORE DE DECISÕES

In [ ]:
# cria e treina o modelo
modelo_dt = DecisionTreeRegressor(
    max_leaf_nodes=6,
    random_state=SEMENTE_ALEATORIA,
)
modelo_dt.fit(normalizador_x.transform(X_treino), normalizador_y.transform(y_treino))

# cria os objetos de figura e eixo
fig, axe = plt.subplots(
    ncols=1,
    nrows=1,
    figsize=(7, 4),
    dpi=150,
)

# plota o grafo da árvore de decisão
tree.plot_tree(
    modelo_dt,
    feature_names=FEATURES,
    ax=axe,
    impurity=False,
    filled=True,
    proportion=True,
    precision=2,
)

# mostra o plot para o usuário
plt.show()